In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import neuronunit
import matplotlib.pyplot as plt
import os
import quantities as pq
from neuronunit.models.reduced import ReducedModel
from scidash_api import rest_client, settings

In [ ]:
from neuronunit.tests.passive import InputResistanceTest
input_resistance_test = InputResistanceTest(
    observation={'mean': 125*pq.MOhm,'std': 40*pq.MOhm,'n': 10},
    name='Input Resistance Test',
    dt=0.000025*pq.ms,
    delay=0.1*pq.ms,
    # padding=0.2*pq.ms,  # we need a fix here when judging the test with the models
    # ration=0.3*pq.ms,  # we need a fix here when judging the test with the models
    amplitude=-1.0000000000000001e-11*pq.pA
)


In [ ]:
model_class_url = 'https://raw.githubusercontent.com/scidash/neuronunit/metacell/neuronunit/models/NeuroML2/LEMS_2007One.xml'

# Instantiate two identical models
models = [ReducedModel(model_class_url, name='Izhikevich_model', backend='jNeuroML') for i in range(2)]

In [ ]:
# Change the parameters of the second model
models[1].set_attrs(**{'izhikevich2007Cell':{'a':'0.04 per_ms'}})
models[1].name = 'Izhikevich_model_2'

In [ ]:
for model in models:
    plt.plot(model.get_membrane_potential(), label=model)
plt.legend();

In [ ]:
s = input_resistance_test.judge(models)

In [ ]:

# use the new SciDash rest api client for uploading the test(s) and model(s) and execute a run


In [ ]:
# create a SciDash rest api client
base_url = settings.get_config('ip')
rest_client = rest_client.ScidashRestApiClient(base_url=base_url, username='admin', password='admin_test')

In [ ]:
# build a new test instance dict object from the test
test_instance = rest_client.build_test_instance_dict(
    test=input_resistance_test,
    name='Name of the Test instance',
    description='Description of the Test instance',
    tags=[]
)

# create a test instance and store it for later
new_test_instance = rest_client.create_test(test_instance=test_instance)

In [ ]:
# build new model instance dict objects from the models
new_model_instances = []
for model in models:
    run_params = {
        "stateVariables":[
            "net1.RS_pop[0].u",
            "net1.RS_pop[0].iSyn",
            "net1.RS_pop[0].iMemb",
            "net1.RS_pop[0].v",
            "net1.explicitInput.RS_Iext.i",
            "time"],
        "watchedVariables":[
            "net1.RS_pop[0].u",
            "net1.RS_pop[0].v"],
        "params":[
            {"name":"Model.neuroml.RS.v0","value":"-60","unit":"mV"},
            {"name":"Model.neuroml.RS.k","value":"0.699999","unit":"nS_per_mV"},
            {"name":"Model.neuroml.RS.vr","value":"-60","unit":"mV"},
            {"name":"Model.neuroml.RS.vt","value":"-40","unit":"mV"},
            {"name":"Model.neuroml.RS.vpeak","value":"35","unit":"mV"},
            {"name":"Model.neuroml.RS.a","value":"0.029999","unit":"per_ms"},
            {"name":"Model.neuroml.RS.b","value":"-2","unit":"nS"},
            {"name":"Model.neuroml.RS.c","value":"-50","unit":"mV"},
            {"name":"Model.neuroml.RS.d","value":"100","unit":"pA"},
            {"name":"Model.neuroml.RS.C","value":"100","unit":"pF"},
            {"name":"Model.neuroml.RS_Iext.delay","value":"0","unit":"ms"},
            {"name":"Model.neuroml.RS_Iext.duration","value":"520","unit":"ms"},
            {"name":"Model.neuroml.RS_Iext.amplitude","value":"100","unit":"pA"}]
    }
    model_instance = rest_client.build_model_instance_dict(
        model=model,
        model_class_url=model_class_url,
        name="Name of the Model instance",
        run_params=run_params,
        tags=[],
        backend=None
    )

    # create the model and store it for later
    new_model_instances.append(rest_client.create_model(model_instance=model_instance))


In [ ]:
# schedule the test and model
# for suites you can pass multiple test and model instances and the name of the new suite
rest_client.schedule(
    suite_name='',
    test_instances=[new_test_instance,],
    model_instances=new_model_instances)
